# Metaphor algorithm

1. Parse file and list objects & mapto implied by subsets, common structures/covers
2. Match structures  
    a. Match types (how to deal with mappable ones? eg. segment->linear)  
    b. Match affected and covers
3. Match representations (keeping the structure consistent)



In [107]:
import yaml
from pprint import pprint

In [110]:
file = 'video-editor.yaml'
with open(file, 'r') as file_handle:
  ex_spec = yaml.safe_load(file_handle)

In [ ]:
def process_listable(target, func):
  if target is None:
    return

  if type(target) == list:
    for target_item in target:
      func(target_item)
  else:
    func(target)

## Parsing for objects

In [ ]:
def register_object(objects, obj):
  # -> means mapto
  # . means subset
  # / means component  # TODO: actually do this!!
  assert(type(obj) is str)

  arrow_array = obj.split('->')
  for arrow_idx, arrow_term in enumerate(arrow_array):
    dot_array = arrow_term.split('.')
    
    for dot_idx, dot_term in enumerate(dot_array):
      subject = '.'.join(dot_array[:dot_idx + 1]) # join up to idx
      objects[subject] = set()
      
      if dot_idx > 0:
        # every sequence maps to the previous element eg. a.b.c => {a.b.c: {a.b}, a.b: {a}}
        previous = '.'.join(dot_array[:dot_idx])
        objects[subject].add(previous)
    
    if arrow_idx > 0:
      # a pair of arrows lhs->rhs => {lhs: {rhs}}
      lhs = objects[arrow_array[arrow_idx - 1]]
      rhs = arrow_array[arrow_idx]
      lhs.add(rhs)

In [ ]:
# Tests
print('testing: a')
test = {}
print('expected:')
pprint({'a': set()})
print('got:')
register_object(test, 'a')
pprint(test)

print()

print('testing: a.b.c')
test = {}
print('expected:')
pprint({'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}})
print('got:')
register_object(test, 'a.b.c')
pprint(test)

print()

print('testing: a->b->c')
test = {}
print('expected:')
pprint({'a': {'b'}, 'b': {'c'}, 'c': set()})
print('got:')
register_object(test, 'a->b->c')
pprint(test)

print()

print('testing: a.b->x->z.w')

test = {}
register_object(test, 'a.b->x->z.w')
print('expected:')
print({'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}})
print('got:')
pprint(test)


testing: a
expected:
{'a': set()}
got:
{'a': set()}

testing: a.b.c
expected:
{'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}}
got:
{'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}}

testing: a->b->c
expected:
{'a': {'b'}, 'b': {'c'}, 'c': set()}
got:
{'a': {'b'}, 'b': {'c'}, 'c': set()}

testing: a.b->x->z.w
expected:
{'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}}
got:
{'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}}


In [ ]:
def register_struct_objects(struct, objects, process_func):
  if struct.get('type') == 'group':
    # Groups also behave as objects, so register them
    name = struct.get('name')
    if name is None:
      print('Warning! No name provided for group. Using `NO NAME PROVIDED` instead. TODO: generate ID.')
      name = 'NO NAME PROVIDED'

    if objects.get(name) is None:
      objects[name] = set()
    
    # TODO: do groups map to their elements? Really, does the transitive property apply? My hunch is no, but need to think more

  process_listable(struct.get('affects'), process_func)
  process_listable(struct.get('covered-by'), process_func)
  # TODO: relate all of the objects affected/covered by a structure. 

  for derivative in struct.get('derivatives', []):
    register_struct_objects(derivative, objects=objects, process_func=process_func)


def register_repr_objects(repr, objects, process_func):
  for repr_obj in repr.get('objects', []):
    assert(len(repr_obj.values()) == 1)
    target_objs = list(repr_obj.values())[0]
    process_listable(target_objs, process_func)


def register_spec_objects(spec, objects={}): # {object: [mapto-targets]}
  # TODO: deal with `objects` block
  def register_object_here(target):
    register_object(objects, target)

  for struct in spec.get('structures', []):
    register_struct_objects(struct, objects, register_object_here)
    
  for repr in spec.get('representations', []):
    register_repr_objects(repr, objects, register_object_here)
  
  return objects

In [ ]:
pprint(register_spec_objects(ex_spec))

{'playhead': {'videos.in-editor/images'},
 'timestamps': set(),
 'tracks': set(),
 'videos': set(),
 'videos.first-frame': {'videos'},
 'videos.in-editor': {'videos'},
 'videos.in-editor/images': {'videos'}}


## Register types